## Assignment 2 Solution

### General

In [21]:
from collections import namedtuple
import numpy as np
import sys, time, pygame
from pygame.locals import *

In [22]:
GameState = namedtuple('GameState', 'to_move, utility, board, moves, depth')

In [23]:
class Game:
    def actions(self, state):
        raise NotImplementedError

    def result(self, state, move):
        raise NotImplementedError

    def utility(self, state, player):
        raise NotImplementedError

    def terminal_test(self, state):
        return not self.actions(state)

    def to_move(self, state):
        return state.to_move

    def display(self, state):
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                if self.terminal_test(state):
                    self.display(state)
                    return self.utility(state, self.to_move(self.initial))

### Tic-Tac-Toe

In [24]:
class TicTacToe(Game):
    """Play TicTacToe on an h x v board, with Max (first player) playing 'X'.
    A state has the player to move, a cached utility, a list of moves in
    the form of a list of (x, y) positions, and a board, in the form of
    a dict of {(x, y): Player} entries, where Player is 'X' or 'O'."""

    def __init__(self, h=3, v=3, k=3):
        self.h = h
        self.v = v
        self.k = k
        moves = [(x, y) for x in range(1, h + 1) for y in range(1, v + 1)]
        self.initial = GameState(to_move='X', utility=0, board={}, moves=moves, depth=0)

    def actions(self, state):
        """Legal moves are any square not yet taken."""
        return state.moves

    def result(self, state, move):
        if move not in state.moves:
            return state  # Illegal move has no effect
        board = state.board.copy()
        board[move] = state.to_move
        moves = list(state.moves)
        moves.remove(move)
        return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves, depth=state.depth+1)

    def utility(self, state, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return state.utility if player == 'X' else -state.utility

    def terminal_test(self, state):
        """A state is terminal if it is won or there are no empty squares."""
        return state.utility != 0 or len(state.moves) == 0

    def display(self, state):
        board = state.board
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                print(board.get((x, y), '.'), end=' ')
            print()
    
    def displayDyn(self, state):
        board = state.board
        res = ""
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                res += str(board.get((x, y), '.'))+' '
            # res+="\n"
        return res

    def compute_utility(self, board, move, player):
        """If 'X' wins with this move, return 1; if 'O' wins return -1; else return 0."""
        if (self.k_in_row(board, move, player, (0, 1)) or
            self.k_in_row(board, move, player, (1, 0)) or
            self.k_in_row(board, move, player, (1, -1)) or
            self.k_in_row(board, move, player, (1, 1))):
            return +1 if player == 'X' else -1
        else:
            return 0

    def k_in_row(self, board, move, player, delta_x_y):
        """Return true if there is a line through move on board for player."""
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n is number of moves in row
        # moving forward
        while board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        # moving backward
        while board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Because we counted move itself twice
        return n >= self.k

# Search Algorithms

### Min Max Search

In [25]:
def minmax_decision(state, game):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. (Figure 5.3)"""

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v
    
    # Body of minmax_decision:
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a)))

### Alpha Beta Search

In [26]:
def alpha_beta_search(state, game):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

### depth limited min max search

In [27]:
def depth_limited_minmax_decision(state, game, maxDepth=5):
    player = game.to_move(state)
        
    startDepth = state.depth
    
    searchTree = dict() # parent: child
    maxDep = -1 # head node is at depth 0
    
    searchTree[game.displayDyn(state)] = dict()

    def max_value(state):
        if (game.terminal_test(state) or ( state.depth-startDepth > maxDepth ) ):
            # print(state.depth, startDepth)
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if (game.terminal_test(state) or ( state.depth-startDepth > maxDepth ) ):
            # print(state.depth, startDepth)
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v
    
    allMoves = game.actions(state)
    result = max(allMoves, key=lambda a: min_value(game.result(state, a)))
    return result

### depth limited alpha beta

In [28]:
def depth_limited_alpha_beta_search(state, game, maxDepth=5):
    player = game.to_move(state)
    
    startDepth = state.depth

    # Functions used by alpha_beta
    def max_value(state, alpha, beta):
        if ( game.terminal_test(state) or ( state.depth-startDepth > maxDepth) ):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if ( game.terminal_test(state) or ( state.depth-startDepth > maxDepth) ):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

### Expermental Varient
- itterative deepening alpha-beta search
- better utility function

In [29]:
def depth_alpha_beta_search(state, game, maxDepth=5):
    player = game.to_move(state)
    
    startDepth = state.depth

    # Functions used by alpha_beta
    def max_value(state, alpha, beta):
        if ( game.terminal_test(state) or ( state.depth-startDepth > maxDepth) ):
            return (game.utility(state, player), state)
        v = (-np.inf, state)
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v, key=lambda a: a[0]) # sort on the basis of utility val
        return v

    def min_value(state, alpha, beta):
        if ( game.terminal_test(state) or ( state.depth-startDepth > maxDepth) ):
            return (game.utility(state, player), state)
        v = (np.inf, state)
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v, key=lambda a: a[0]) # sort on the basis of utility val
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    best_state = state
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta)
        if v[0] > best_score:
            best_score, best_state = v[0], v[1]
            best_action = a
    return (best_action, best_state)

def itterative_deepening_alpha_beta_search(state, game):
    result = state
    game.display(result)
    

In [30]:
l = (0,1)
d = (1,-4)
print(max(l,d, key=lambda a: a[1]))

(0, 1)


### play implimentation

In [31]:
def readUserInput(board, x_r=3, y_r=3):
    try:
        (x, y) = tuple(map(int, str(input()).split()))
        while ( x>x_r or x<1 or y>y_r or y<1 or (x,y) in board.keys() ):
            print("invalid input please try again")
            (x, y) = tuple(map(int, str(input()).split()))
        return (x, y)
    except:
        print("invalid input format, try again")

In [32]:
def playGame(decision_algo, x_r=3, y_r=3):
    sampStart = TicTacToe(x_r, y_r)

    ittr = x_r*y_r # total Number of moves possible
    while(ittr>0):
        if (ittr!=0):
            print('User (X)')
            ittr-=1
            # User move
            NewUserMove = readUserInput(sampStart.initial.board, x_r, y_r)
            NewUserState = sampStart.result(sampStart.initial, NewUserMove)
            moveBy = sampStart.initial.to_move
            
            sampStart.display(NewUserState)
            print("state depth : ",NewUserState.depth)
            sampStart.initial = NewUserState
            decision = sampStart.compute_utility(
                sampStart.initial.board,
                NewUserMove,
                moveBy
            )
            if ( decision ):
                print(decision,'user wins')
                break
            print()

        if (ittr!=0):
            start = time.time()
            print("computer (0)")
            ittr-=1
            # Computer move
            NewMove = decision_algo(sampStart.initial, sampStart)
            NewState = sampStart.result(sampStart.initial, NewMove)
            moveBy = sampStart.initial.to_move
            print("time to compute : ",time.time()-start)
            sampStart.display(NewState)
            print("state depth : ",NewState.depth)
            sampStart.initial = NewState
            decision = sampStart.compute_utility(
                sampStart.initial.board,
                NewMove,
                moveBy
            )
            if ( decision ):
                print(decision,'computer wins')
                break
            print()

    if (ittr==0):
        print("Draw match. Nice play")
    print("-----------------------------")

## Interactive Play

### Play Game

In [73]:
def playVis(x_l=3, y_l=3,decision_algo=alpha_beta_search):
    # x_l row count, y_l column count
    
    sampStart = TicTacToe(y_l, x_l)
    
    BLACK = (0, 0, 0)
    WHITE = (255, 255, 255)
    GREEN = (0, 255, 0)
    RED = (255, 0, 0)
    
    gridColor = [GREEN, RED]
    colorInd = 1
    
    # This sets the WIDTH and HEIGHT of each grid location
    WIDTH = 40
    HEIGHT = 40
    
    # to end process when there is not more move left
    moveCount = 0
    totalCount = x_l*y_l
    
    # This sets the margin between each cell
    MARGIN = 5

    # Create a 2 dimensional array. A two dimensional
    # array is simply a list of lists.
    grid = [[-1 for x in range(x_l)] for y in range(y_l)]

    # grid[i][j]=0 for green grid[i][j]=1 for red else -1         

    # Initialize pygame
    pygame.init()

    # Set the HEIGHT and WIDTH of the screen
    WINDOW_SIZE = [ (WIDTH+MARGIN)*x_l + 5 , (HEIGHT+MARGIN)*y_l + 5]
    screen = pygame.display.set_mode(WINDOW_SIZE)

    # Set title of screen
    pygame.display.set_caption("Tic Tac Toe")

    # Loop until the user clicks the close button.
    done = False

    # Used to manage how fast the screen updates
    clock = pygame.time.Clock()
    
    # playerTurn = 0  # user, 1 for computer
    
    playerTurn=1
    colorInd=1
    
    # Set the screen background
    screen.fill(BLACK)
    
    for row in range(y_l):
        for column in range(x_l):
            color = WHITE
            if grid[row][column] != -1:
                color = gridColor[ grid[row][column] ]
            pygame.draw.rect(
                screen,
                color,
                [
                    (MARGIN + WIDTH) * column + MARGIN,
                    (MARGIN + HEIGHT) * row + MARGIN,
                    WIDTH, HEIGHT
                ]
            )
    pygame.display.update()
    
    while not done:      
        invMove = False
        if (moveCount >= totalCount):
            done = True
            break
        for event in pygame.event.get():  # User did something
            if event.type == pygame.QUIT:  # If user clicked close
                done = True
            elif event.type == pygame.MOUSEBUTTONDOWN:
                invMove = False

                ## user move
                # User clicks the mouse. Get the position
                pos = pygame.mouse.get_pos()

                # Change the x/y screen coordinates to grid coordinates
                column = pos[0] // (WIDTH + MARGIN)
                row = pos[1] // (HEIGHT + MARGIN)
                if ( (row+1, column+1) in sampStart.initial.board.keys() ):
                    pygame.display.set_caption("Invalid move")
                    print("invalid move found")
                    invMove = True

                else:
                    #-------------- User --------------------
                    
                    colorInd = (colorInd+1)%2
                    playerTurn = (playerTurn+1)%2
                    
                    moveCount += 1
                    NewUserMove = ( row+1, column+1 )
                    NewUserState = sampStart.result(sampStart.initial, NewUserMove)
                    moveBy = sampStart.initial.to_move      
                    sampStart.display(NewUserState)
                    sampStart.initial = NewUserState
                    decision = sampStart.compute_utility(
                        sampStart.initial.board,
                        NewUserMove,
                        moveBy
                    )
                    
                    # Set that location to one
                    grid[row][column] = colorInd
                    print("Click ", pos, "Grid coordinates: ", row, column)
                    for row in range(y_l):
                        for column in range(x_l):
                            color = WHITE
                            if grid[row][column] != -1:
                                color = gridColor[ grid[row][column] ]
                            pygame.draw.rect(
                                screen,
                                color,
                                [
                                    (MARGIN + WIDTH) * column + MARGIN,
                                    (MARGIN + HEIGHT) * row + MARGIN,
                                    WIDTH, HEIGHT
                                ]
                            )
                    pygame.display.update()

                    if ( decision ):
                        print(decision,'user wins')
                        pygame.display.set_caption("User Win")
                        time.sleep(2)
                        done = True
                        break
                    print()
                    
                    # --------- Chenk for no more moves left ---------
                    if (moveCount >= totalCount):
                        print(decision,'Tie')
                        pygame.display.set_caption("Draw match")
                        time.sleep(2)
                        done = True
                        break
                    
                    # ----------- Computer -------------------
                    
                    colorInd = (colorInd+1)%2
                    playerTurn = (playerTurn+1)%2
                    
                    ## computer starts calculating its move
                    moveCount += 1
                    # get from agent
                    print("comuter loading")
                    NewMove = decision_algo(sampStart.initial, sampStart)
                    print("results found")
                    row = NewMove[0]-1
                    column = NewMove[1]-1
                    if ( (row+1, column+1) in sampStart.initial.board.keys() ):
                        pygame.display.set_caption("Invalid move")
                        print("invalid move found")
                        invMove = True

                    pos = (row, column)
                    NewState = sampStart.result(sampStart.initial, NewMove)
                    moveBy = sampStart.initial.to_move
                    sampStart.display(NewState)
                    sampStart.initial = NewState
                    decision = sampStart.compute_utility(
                        sampStart.initial.board,
                        NewMove,
                        moveBy
                    )
                    
                    # Set that location to one
                    grid[row][column] = colorInd
                    print("Click ", pos, "Grid coordinates: ", row, column)
                    for row in range(y_l):
                        for column in range(x_l):
                            color = WHITE
                            if grid[row][column] != -1:
                                color = gridColor[ grid[row][column] ]
                            pygame.draw.rect(
                                screen,
                                color,
                                [
                                    (MARGIN + WIDTH) * column + MARGIN,
                                    (MARGIN + HEIGHT) * row + MARGIN,
                                    WIDTH, HEIGHT
                                ]
                            )
                    pygame.display.update()
                    
                    if ( decision ):
                        print(decision,'computer wins')
                        pygame.display.set_caption("Computer Win")
                        time.sleep(2)
                        done = True
                    print()
    pygame.quit()
            
playVis(4,4)

. X . . 
. . . . 
. . . . 
. . . . 
Click  (50, 25) Grid coordinates:  0 1

comuter loading
results found
O X . . 
. . . . 
. . . . 
. . . . 
Click  (0, 0) Grid coordinates:  0 0

O X . . 
. . X . 
. . . . 
. . . . 
Click  (124, 66) Grid coordinates:  1 2

comuter loading
results found
O X O . 
. . X . 
. . . . 
. . . . 
Click  (0, 2) Grid coordinates:  0 2

O X O X 
. . X . 
. . . . 
. . . . 
Click  (159, 32) Grid coordinates:  0 3

comuter loading
results found
O X O X 
O . X . 
. . . . 
. . . . 
Click  (1, 0) Grid coordinates:  1 0

O X O X 
O . X . 
X . . . 
. . . . 
Click  (41, 106) Grid coordinates:  2 0

comuter loading
results found
O X O X 
O O X . 
X . . . 
. . . . 
Click  (1, 1) Grid coordinates:  1 1

O X O X 
O O X . 
X X . . 
. . . . 
Click  (68, 111) Grid coordinates:  2 1
1 user wins

comuter loading
results found
O X O X 
O O X . 
X X O . 
. . . . 
Click  (2, 2) Grid coordinates:  2 2
-1 computer wins



In [72]:
pygame.quit()

In [30]:
def playVisualGrid(x_l=3, y_l=3,decision_algo=alpha_beta_search):
    
    # x_l row count, y_l column count
    
    sampStart = TicTacToe(y_l, x_l)
    
    BLACK = (0, 0, 0)
    WHITE = (255, 255, 255)
    GREEN = (0, 255, 0)
    RED = (255, 0, 0)
    
    gridColor = [GREEN, RED]
    colorInd = 1
    
    # This sets the WIDTH and HEIGHT of each grid location
    WIDTH = 40
    HEIGHT = 40
    
    # to end process when there is not more move left
    moveCount = 0
    totalCount = x_l*y_l
    
    # This sets the margin between each cell
    MARGIN = 5

    # Create a 2 dimensional array. A two dimensional
    # array is simply a list of lists.
    grid = [[-1 for x in range(x_l)] for y in range(y_l)]

    # grid[i][j]=0 for green grid[i][j]=1 for red            

    # Initialize pygame
    pygame.init()

    # Set the HEIGHT and WIDTH of the screen
    WINDOW_SIZE = [ (WIDTH+MARGIN)*x_l + 5 , (HEIGHT+MARGIN)*y_l + 5]
    screen = pygame.display.set_mode(WINDOW_SIZE)

    # Set title of screen
    pygame.display.set_caption("Tic Tac Toe")

    # Loop until the user clicks the close button.
    done = False

    # Used to manage how fast the screen updates
    clock = pygame.time.Clock()
    
    # playerTurn = 0  # user, 1 for computer
    
    playerTurn=1
    colorInd=1
    
    # Set the screen background
    screen.fill(BLACK)
    
    # -------- Main Program Loop -----------
    while not done:      
        invMove = False
        if (moveCount >= totalCount):
            done = True
            break
        for event in pygame.event.get():  # User did something
            if event.type == pygame.QUIT:  # If user clicked close
                done = True
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if not invMove:
                    colorInd = (colorInd+1)%2
                    playerTurn = (playerTurn+1)%2
                invMove = False
                
                if (playerTurn==0):
                    # User clicks the mouse. Get the position
                    pos = pygame.mouse.get_pos()

                    # Change the x/y screen coordinates to grid coordinates
                    column = pos[0] // (WIDTH + MARGIN)
                    row = pos[1] // (HEIGHT + MARGIN)
                    if ( (row+1, column+1) in sampStart.initial.board.keys() ):
                        pygame.display.set_caption("Invalid move")
                        print("invalid move found")
                        invMove = True
                        
                    else:
                        moveCount += 1
                        NewUserMove = ( row+1, column+1 )
                        NewUserState = sampStart.result(sampStart.initial, NewUserMove)
                        moveBy = sampStart.initial.to_move      
                        sampStart.display(NewUserState)
                        sampStart.initial = NewUserState
                        decision = sampStart.compute_utility(
                            sampStart.initial.board,
                            NewUserMove,
                            moveBy
                        )
                        
                        if ( decision ):
                            print(decision,'user wins')
                            pygame.display.set_caption("User Win")
                            time.sleep(2)
                            done = True
                            # pygame.quit()
                        print()
                
                else:
                    moveCount += 1
                    # get from agent
                    print("comuter loading")
                    NewMove = decision_algo(sampStart.initial, sampStart)
                    print("results found")
                    row = NewMove[0]-1
                    column = NewMove[1]-1
                    if ( (row+1, column+1) in sampStart.initial.board.keys() ):
                        pygame.display.set_caption("Invalid move")
                        print("invalid move found")
                        invMove = True
                    
                    pos = (row, column)
                    NewState = sampStart.result(sampStart.initial, NewMove)
                    moveBy = sampStart.initial.to_move
                    sampStart.display(NewState)
                    sampStart.initial = NewState
                    decision = sampStart.compute_utility(
                        sampStart.initial.board,
                        NewMove,
                        moveBy
                    )
                        
                    if ( decision ):
                        print(decision,'computer wins')
                        pygame.display.set_caption("Computer Win")
                        time.sleep(2)
                        done = True
                    print()
                
                # Set that location to one
                grid[row][column] = colorInd
                print("Click ", pos, "Grid coordinates: ", row, column)
        
        if not invMove:
            # Draw the grid
            for row in range(y_l):
                for column in range(x_l):
                    color = WHITE
                    if grid[row][column] != -1:
                        color = gridColor[ grid[row][column] ]
                    pygame.draw.rect(
                        screen,
                        color,
                        [
                            (MARGIN + WIDTH) * column + MARGIN,
                            (MARGIN + HEIGHT) * row + MARGIN,
                            WIDTH, HEIGHT
                        ]
                    )

        # Limit to 60 frames per second
        clock.tick(60)

        # Go ahead and update the screen with what we've drawn.
        pygame.display.flip()

    # Be IDLE friendly. If you forget this line, the program will 'hang'
    # on exit.
    pygame.quit()

    
playVisualGrid(4,3)

X . . . 
. . . . 
. . . . 

Click  (24, 14) Grid coordinates:  0 0
comuter loading
results found
X O . . 
. . . . 
. . . . 

Click  (0, 1) Grid coordinates:  0 1
X O . . 
X . . . 
. . . . 

Click  (17, 59) Grid coordinates:  1 0
comuter loading
results found
X O O . 
X . . . 
. . . . 

Click  (0, 2) Grid coordinates:  0 2
X O O . 
X . . . 
X . . . 
1 user wins

Click  (32, 114) Grid coordinates:  2 0


In [17]:
pygame.quit()

In [ ]:
def playGameMinMax():
    sampStart = TicTacToe()

    ittr = 9
    while(ittr>0):
        if (ittr!=0):
            print('User (X)')
            ittr-=1
            # User move
            NewUserMove = readUserInput(sampStart.initial.board)
            NewUserState = sampStart.result(sampStart.initial, NewUserMove)
            moveBy = sampStart.initial.to_move
            
            sampStart.display(NewUserState)
            sampStart.initial = NewUserState
            decision = sampStart.compute_utility(sampStart.initial.board,NewUserMove,moveBy)
            if ( decision ):
                print('user wins')
                break
            print()

        if (ittr!=0):
            print("computer (0)")
            ittr-=1
            # Computer move
            NewMove = minmax_decision(sampStart.initial, sampStart)
            NewState = sampStart.result(sampStart.initial, NewMove)
            moveBy = sampStart.initial.to_move

            sampStart.display(NewState)
            sampStart.initial = NewState
            decision = sampStart.compute_utility(sampStart.initial.board,NewMove,moveBy)
            if ( decision ):
                print('computer wins')
                break
            print()

    if (ittr==0):
        print("Draw match. Nice play")

# or
playGame(minmax_decision)

In [ ]:
def playGameAlphaBeta():
    sampStart = TicTacToe()

    ittr = 9
    while(ittr>0):
        if (ittr!=0):
            print('User (X)')
            ittr-=1
            # User move
            NewUserMove = readUserInput(sampStart.initial.board)
            NewUserState = sampStart.result(sampStart.initial, NewUserMove)
            moveBy = sampStart.initial.to_move
            
            sampStart.display(NewUserState)
            sampStart.initial = NewUserState
            decision = sampStart.compute_utility(sampStart.initial.board,NewUserMove,moveBy)
            if ( decision ):
                print('user wins')
                break
            print()

        if (ittr!=0):
            print("computer (0)")
            ittr-=1
            # Computer move
            NewMove = alpha_beta_search(sampStart.initial, sampStart)
            NewState = sampStart.result(sampStart.initial, NewMove)
            moveBy = sampStart.initial.to_move
            
            sampStart.display(NewState)
            sampStart.initial = NewState
            decision = sampStart.compute_utility(sampStart.initial.board,NewMove,moveBy)
#             print(sampStart.initial.board)
            if ( decision ):
                print('computer wins')
                break
            print()

    if (ittr==0):
        print("Draw match. Nice play")
# playGame(alpha_beta_search)

In [ ]:
# playGame(depth_limited_alpha_beta_search, 5, 5)